# OpenEO: Sentinel 1 

Connection to VITO backend for cloud computing

In [ ]:
import openeo
import geopandas as gpd
from pathlib import Path
import numpy as np
import pickle
import xarray as xr
import hvplot.xarray
import os
connection = openeo.connect("openeo.vito.be").authenticate_oidc()
pad = Path(os.getcwd())
if pad.name != "Python":
    pad_correct = Path("../../Python")
    os.chdir(pad_correct)

Load in the Shapefile of the Zwalm in EPSG:4326

In [ ]:
shape_zwalm = gpd.read_file('data/Zwalm_shape/zwalm_shapefile_emma.shp')
shape_zwalm.plot()
extent = shape_zwalm.total_bounds
print(extent)

In [ ]:
#extent = [3.66751526, 50.76325563, 3.83821038, 50.90341411]

In [ ]:
#connection.list_collections()

In [ ]:
#connection.describe_collection('SENTINEL1_GRD')
connection.describe_collection('S1_GRD_SIGMA0_ASCENDING')

In [ ]:
collection = 'SENTINEL1_GRD' #Ground Range Detected
spatial_extent = {'west':extent[0],'east':extent[2],'south':extent[1],'north':extent[3]}
temporal_extent = ["2014-11-18", "2022-11-05"]  #start with limited extent as a test
bands = ["VV"] #enkel in deze geïnteresseerd 
properties = {
    "sat:orbit_state": lambda od: od == "ASCENDING", ##filter on ascending vs descending
    "sar:instrument_mode":lambda mode: mode == "IW" ## Orbit direction filtering
}
s1_a = connection.load_collection(
    collection_id = collection,
    spatial_extent= spatial_extent,
    temporal_extent = temporal_extent,
    bands = bands,
    properties = properties
)
s1_a = s1_a.ard_normalized_radar_backscatter(elevation_model = "COPERNICUS_30")
s1_a_mask = s1_a.mask_polygon(shape_zwalm['geometry'].values[0])
#from linear to db
s1_a_mask = s1_a_mask.apply(lambda x:10 * x.log(base=10))

Now make a job to send to the cloud

In [ ]:
# job_s1a = s1_a_mask.create_job(title = "S1-A")
# job_id_s1a = job_s1a.job_id
# if job_id_s1a:
#     print("Batch job created with id: ",job_id_s1a)
#     job_s1a.start_job()
# else:
#     print("Error! Job ID is None")

In [ ]:
# job_reconnect = connection.job(job_id_s1a)
# job_description = job_reconnect.describe_job()
# print("Batch job with id: ",job_id_s1a, ' is ',job_description['status'])

## Attempt at splitting requests per year!

In [ ]:
temporal_extent = ["2014-11-18", "2022-11-05"]  

In [ ]:
list_temp_extent = []
list_temp_extent.append([temporal_extent[0],"2014-12-31"])
import numpy as np
for year in np.arange(2015,2023):
    if year == 2022:
        #print([str(year)+"-01-01",temporal_extent[1]])
        list_temp_extent.append([str(year)+"-01-01",temporal_extent[1]])
    else:
        #print([str(year)+"-01-01",str(year)+ "-12-31"])
        list_temp_extent.append([str(year)+"-01-01",str(year)+ "-12-31"])
print(list_temp_extent)

In [ ]:
years = np.arange(2014,2023)
job_title_list = []
job_id_list = []
for i, temporal_extent in enumerate(list_temp_extent):
    s1_a = connection.load_collection(
        collection_id = collection,
        spatial_extent= spatial_extent,
        temporal_extent = temporal_extent,
        bands = bands,
        properties = properties
    )
    s1_a = s1_a.ard_normalized_radar_backscatter(elevation_model = "COPERNICUS_30")
    s1_a_mask = s1_a.mask_polygon(shape_zwalm['geometry'].values[0])
    #from linear to db
    #s1_a_mask = s1_a_mask.apply(lambda x:10 * x.log(base=10))
    job_title = "S1-A-" +  str(years[i])
    job_title_list.append(job_title)
    job_s1a = s1_a_mask.create_job(title = job_title, out_format= 'NetCDF')
    job_id_s1a = job_s1a.job_id
    if job_id_s1a:
        print("Batch job created with id: ",job_id_s1a)
        job_s1a.start_job()
        job_id_list.append(job_id_s1a)
    else:
        print("Error! Job ID is None")

In [ ]:
with open('s1_a_job_id_list.pickle', 'wb') as handle:
    pickle.dump(job_id_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
job_id_list = pickle.load(open("temp/s1_job_id_list.pickle", "rb"))
for i,job_id in enumerate(job_id_list):
    job_connection = connection.job(job_id)
    results = job_connection.get_results()
    name_netcdf = job_title_list[i] + '.nc'
    filepath = "data/LAI/" + name_netcdf
    print(filepath)
    results.download_file(filepath)

In [ ]:
# i = 2
# temporal_extent = list_temp_extent[i]


In [ ]:
# s1_a = connection.load_collection(
#         collection_id = collection,
#         spatial_extent= spatial_extent,
#         temporal_extent = temporal_extent,
#         bands = bands,
#         properties = properties
#     )
# s1_a = s1_a.ard_normalized_radar_backscatter(elevation_model = "COPERNICUS_30")
# s1_a_mask = s1_a.mask_polygon(shape_zwalm['geometry'].values[0])
# #from linear to db
# s1_a_mask = s1_a_mask.apply(lambda x:10 * x.log(base=10))
# job_title = "S1-A-" +  str(years[i])
# job_title_list.append(job_title)
# job_s1a = s1_a_mask.create_job(title = job_title, out_format='NetCDF')
# job_id_s1a = job_s1a.job_id
# if job_id_s1a:
#     print("Batch job created with id: ",job_id_s1a)
#     job_s1a.start_job()
#     job_id_list.append(job_id_s1a)
# else:
#     print("Error! Job ID is None")

## Downloading the results test

In [ ]:
# job_2015 = connection.job( "j-59c6be40a6fd44cab419355802e14832")
# results = job_2015.get_results()
# results.download_files("data/g0_OpenEO/S1_A_2015.nc")